In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm #Configuration parameters
epochs = 100 # Number of training epochs
batch_size = 64 # Size of each training batch
sample_size = 100 # Size of the random noise vector input to generator
g_lr = 1.0e-4 # Learning rate for the generator
d_lr = 1.0e-4 # Learning rate for the discriminator
#DataLoader for the MNIST dataset

transform = transforms.ToTensor()
dataset = datasets.MNIST (root='./data', train=True, download=True, transform=transform) # fix: assign to dataset
dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True)
#Generator Network
class Generator(nn.Sequential):
    def __init__(self, sample_size: int): # fix: __init
    # The generator takes a random noise vector of size sample size and outputs a 784-dimensional image vec
        super().__init__(
            nn.Linear(sample_size, 128), # First linear layer
            nn.LeakyReLU(0.01), # Activation
            nn.Linear(128,784), # Output layer to match image size
            nn.Sigmoid()) # Normalize outputs between 0 and 1
        self.sample_size = sample_size
    def forward(self, batch_size: int):
        z = torch.randn(batch_size, self.sample_size) # Generate random noise input
        output = super().forward(z) # Forward pass
        generated_images = output.reshape(batch_size, 1, 28, 28) # Reshape to image format
        return generated_images
# Discriminator Network
class Discriminator(nn.Sequential):
    def __init__(self): # fix: __init
        # The discriminator takes a 784-dimensional image vector and outputs a single logit
        super().__init__(
            nn.Linear(784, 128), # Input layer
            nn.LeakyReLU(0.01), # Activation
            nn.Linear(128, 1)) # Output logit
    def forward(self, images: torch.Tensor, targets: torch.Tensor):
        prediction = super().forward(images.reshape(-1, 784))   #Flatten image and forward pass
        loss = F.binary_cross_entropy_with_logits(prediction, targets) # BCE loss with logits
        return loss
#Function to save generated images in a grid layout
def save_image_grid(epoch: int, images: torch.Tensor, ncol: int):
    image_grid = make_grid(images, ncol) #Arrange images into a grid
    image_grid = image_grid.permute(1, 2, 0) #Convert from CHW to HWC for matplotlib # fix: assign to image_grid
    image_grid = image_grid.cpu().numpy() # Convert to numpy for plotting # Moved this line inside the function
    plt.imshow(image_grid) # Display image grid
    plt.xticks([]); plt.yticks([]) # Remove axis ticks
    plt.savefig(f'generated_{epoch:03d}.jpg') # Save image
    plt.close() #close the plot
#Labels for training the discriminator
real_targets = torch.ones(batch_size, 1) # Label for real images
fake_targets = torch.zeros(batch_size, 1) # Label for fake images
# Initialize generator and discriminator
generator = Generator(sample_size)
discriminator = Discriminator()
# Optimizers
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=d_lr) # fix: corrected d_1r to d_lr
g_optimizer = torch.optim.Adam(generator.parameters(), lr=g_lr)
# Training loop
for epoch in range(epochs):
    d_losses = [] # Track discriminator losses
    g_losses = [] # Track generator losses
    for images, labels in tqdm(dataloader):
        #
        #Train the Discriminator Network
        #
        discriminator.train() # Set discriminator to training mode
        d_loss = discriminator(images, real_targets) # Loss for real images # fix: assign to d_loss, call discriminator
        generator.eval() # Set generator to eval mode for generating fakes
        with torch.no_grad():
            generated_images = generator(batch_size) # Generate fake images
        d_loss += discriminator(generated_images, fake_targets) # fix: call discriminator
        d_optimizer.zero_grad() # Clear gradients
        d_loss.backward() # Backpropagate
        d_optimizer.step() # Update discriminator parameters
        #Train the Generator Network
        #
        generator.train() # Set generator to training mode
        generated_images = generator(batch_size) # Generate fake images
        discriminator.eval() # Discriminator in eval mode but still requires gradients
        g_loss = discriminator(generated_images, real_targets) # Try to fool discriminator # fix: call discriminator
        g_optimizer.zero_grad() # Clear gradients
        g_loss.backward() # Backpropagate
        g_optimizer.step() # Update generator parameters
        # Record losses
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())
    # Print losses at end of each epoch
    print(epoch, np.mean(d_losses), np.mean(g_losses))
    # Save generated samples
    save_image_grid(epoch, generator(batch_size), ncol=8)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 487kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.3MB/s]
100%|██████████| 937/937 [00:11<00:00, 81.03it/s]


0 0.42999258987295463 2.720147139680042


100%|██████████| 937/937 [00:10<00:00, 87.22it/s]


1 0.32672598528632996 2.2346837675813167


100%|██████████| 937/937 [00:11<00:00, 84.00it/s]


...

97 0.7867777892085251 1.912967690281578


100%|██████████| 937/937 [00:11<00:00, 83.70it/s]


98 1.9596027783421341 1.1431519975339082


100%|██████████| 937/937 [00:11<00:00, 83.06it/s]

99 3.164161084174092 0.6676997320374659
